### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import psycopg2

### Connection (URL, DB) / Create Table / Import

In [2]:
def connect_and_save_data(matchday, team_name, pass_percentage):
    try:
        connection = psycopg2.connect(
            dbname='PSQL_ADSFS2023Gruppe15',
            user='ADSFS2023Gruppe15',
            password='ADS_FS_2023_G15!?',
            host='localhost',
            port='5432'
        )

        cursor = connection.cursor()

        # Füge die extrahierten Daten in die Tabelle ein
        insert_data_query = '''
            INSERT INTO bundesliga_pass_stats (matchday, team_name, pass_percentage) VALUES (%s, %s, %s);
        '''
        cursor.execute(insert_data_query, (matchday, team_name, float(pass_percentage.replace(',', '.'))))
        connection.commit()

    except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
    finally:
        # Schließe die Verbindung
        if connection:
            cursor.close()
            connection.close()
            

# Durchlaufe die Spieltage (md1 bis md34) und lade die Daten für jeden Spieltag
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-paesse/'

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Suche nach allen Mannschaftsnamen und Passquoten
        team_elements = soup.find_all('td', class_='team-name team-name-list')
        pass_percentage_elements = soup.find_all('td', class_='team_stats-passes_complete_percentage team_stats-passes_complete_percentage-format')

        # Iteriere über die gefundenen Elemente und extrahiere die gewünschten Informationen
        for team_element, pass_percentage_element in zip(team_elements, pass_percentage_elements):
            team_name = team_element.text.strip()
            pass_percentage = pass_percentage_element.text.strip()

            # Speichere die Daten in der PostgreSQL-Datenbank
            connect_and_save_data(matchday, team_name, pass_percentage)

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday}')